# Train your own Model with Nerf Studio from Video

Author: Deeepwin  
Date: 18.03.2023 
***

## Installation

This installation instructions explains in how to install Nerfstudio in a Conda enviroment that has a separte CUDA installtion. It makes the installation of Nerfstudio a bit more tricky. In my case I have installed CUDA version 11.3 in Conda which I want to use for this project. The Conda environment is called `points`. On the Ubuntu installation I run CUDA 12.1.

### Checking CUDA Installation

Make sure you have installed CUDA toolkit in Conda.

In [ ]:
!conda install -c "nvidia/label/cuda-11.3.1" cuda-toolkit
!conda install -c "nvidia/label/cuda-11.3.1" cuda-nvcc

In [ ]:
# check CUDA versions, must be 11.3.1
!conda list cudatoolkit
!conda list cudnn
!conda list cuda-nvcc

In [ ]:
# check Pytorch version, must be 1.13.1
torch_version = !pip show torch
torch_version[1]

Pytorch automatically installs nvidia_cublas_cu11, which will cause an error in the installation of Nerfstudio. That's why we need to remove it here.

In [ ]:
!pip uninstall -y nvidia_cublas_cu11

### Setup Tiny CUDA NN

For CUDA version 11.3 you need g++ version 9 (max version is 10.2.1), see [here](https://stackoverflow.com/questions/6622454/cuda-incompatible-with-my-gcc-version). General instructions in how to setup different versions of g++ can be found [here](https://tuxamito.com/wiki/index.php/Installing_newer_GCC_versions_in_Ubuntu).

In [ ]:
!sudo apt -y install g++-9
!sudo update-alternatives --install /usr/bin/g++ g++ /usr/bin/g++-9 9
!sudo update-alternatives --config g++
!g++ --version


In [ ]:
!sudo apt -y install gcc-9
!sudo update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-9 9
!sudo update-alternatives --config gcc
!gcc --version

Now set CUDA_HOME enviroment variable. Otherwise you cannot install `tiny-cuda-nn`. Update path according to your Conda installation.

In [ ]:
!conda env config vars set CUDA_HOME="/home/martin/anaconda3/envs/points/lib/"
!conda env config vars list

Install `tiny-cuda-nn`

In [ ]:
!pip install git+https://github.com/NVlabs/tiny-cuda-nn/#subdirectory=bindings/torch

Finally install Nerfstudio see [website](https://docs.nerf.studio/en/latest/quickstart/installation.html).

In [ ]:
%%capture
!pip install -y nerfstudio

### Install Tools

In [ ]:
!sudo apt install colmap

## Train First Model

In [ ]:
%%capture
!ns-download-data nerfstudio --capture-name=poster

Lets train.

In [ ]:
!ns-train nerfacto --data data/nerfstudio/poster

To export as point cloud run:

In [ ]:
%%capture
!ns-export pointcloud --load-config outputs/poster/nerfacto/2023-03-18_122407/config.yml --output-dir exports/pcd/ --num-points 1000000 --remove-outliers True --estimate-normals False --use-bounding-box True --bounding-box-min -1 -1 -1 --bounding-box-max 1 1 1

In [ ]:
%%capture

# show  point cloud in separate window
!open3d draw exports/pcd/point_cloud.ply 

In [ ]:
# as python code
if False:
    import open3d as o3d

    pcd = o3d.io.read_point_cloud("exports/pcd/point_cloud.ply")
    o3d.visualization.draw_geometries([pcd])

## Using own Data

If you have no phone with LiDAR like me, install [Kiri Engine](https://www.kiriengine.com/) app. Follow the instructions [here](https://docs.nerf.studio/en/latest/quickstart/custom_dataset.html). Download .zip file, unzip and place in `export` folder. Afterwards run:

In [3]:
!ns-train nerfacto --data data/mine/mouse --viewer.websocket-port=7008

Step (% Done)       Train Iter (time)    ETA (time)           Train Rays / Sec     Vis Rays / Sec        
-------------------------------------------------------------------------------------------------------- 
8920 (29.73%)       23.087 ms            8 m, 6 s             181.65 K                                   
8930 (29.77%)       23.450 ms            8 m, 14 s            179.77 K                                   
8940 (29.80%)       22.912 ms            8 m, 2 s             183.05 K                                   
8950 (29.83%)       22.880 ms            8 m, 1 s             183.72 K                                   
8960 (29.87%)       23.594 ms            8 m, 16 s            178.94 K                                   
8970 (29.90%)       23.323 ms            8 m, 10 s            179.55 K                                   
8980 (29.93%)       22.426 ms            7 m, 51 s            185.73 K                                   
8990 (29.97%)       23.157 ms            8 m, 

Open viewer and monitor progress. Once training is complete, go to expert and copy export command to field below:

In [ ]:
!ns-export pointcloud --load-config outputs/poster/nerfacto/2023-03-18_122407/config.yml --output-dir exports/pcd/ --num-points 1000000 --remove-outliers True --estimate-normals False --use-bounding-box True --bounding-box-min -1 -1 -1 --bounding-box-max 1 1 1

In [ ]:
%%capture

# show  point cloud in separate window
!open3d draw exports/mouse/point_cloud.ply 